In [1]:
import keras
import numpy as np
import pandas as pd
import os
from keras.preprocessing import image
import tensorflow as tf
from PIL import Image
from sklearn.metrics import accuracy_score
import re


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input, EfficientNetB0

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.preprocessing import image

In [5]:
df=pd.read_csv('df_f.csv')
df

,Image,Category
0,battery/battery1.jpg,0
1,battery/battery10.jpg,0
2,battery/battery100.jpg,0
3,battery/battery101.jpg,0
4,battery/battery102.jpg,0
...,...,...
15510,white-glass/white-glass95.jpg,12
15511,white-glass/white-glass96.jpg,12
15512,white-glass/white-glass97.jpg,12
15513,white-glass/white-glass98.jpg,12


In [6]:
garbage_types_labels = {
    0: 'battery',
    1: 'biological',
    3: 'brown-glass',
    4: 'cardboard',
    5: 'clothes',
    6: 'green-glass',
    7: 'metal',
    8: 'paper',
    9: 'plastic',
    10: 'shoes',
    11: 'trash',
    12: 'white-glass'
}

In [7]:
df

,Image,Category
0,battery/battery1.jpg,0
1,battery/battery10.jpg,0
2,battery/battery100.jpg,0
3,battery/battery101.jpg,0
4,battery/battery102.jpg,0
...,...,...
15510,white-glass/white-glass95.jpg,12
15511,white-glass/white-glass96.jpg,12
15512,white-glass/white-glass97.jpg,12
15513,white-glass/white-glass98.jpg,12


In [8]:
df["Category"] = df["Category"].replace(garbage_types_labels) 

In [9]:
df

,Image,Category
0,battery/battery1.jpg,battery
1,battery/battery10.jpg,battery
2,battery/battery100.jpg,battery
3,battery/battery101.jpg,battery
4,battery/battery102.jpg,battery
...,...,...
15510,white-glass/white-glass95.jpg,white-glass
15511,white-glass/white-glass96.jpg,white-glass
15512,white-glass/white-glass97.jpg,white-glass
15513,white-glass/white-glass98.jpg,white-glass


In [10]:
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.5, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train = train_df.shape[0]
validate = validate_df.shape[0]
test=test_df.shape[0]

In [11]:
test

1552

In [12]:
validate

1551

In [13]:
train

12412

In [14]:
train_df

,Image,Category
0,clothes/clothes2288.jpg,clothes
1,clothes/clothes954.jpg,clothes
2,cardboard/cardboard846.jpg,cardboard
3,plastic/plastic487.jpg,plastic
4,shoes/shoes551.jpg,shoes
...,...,...
12407,clothes/clothes2586.jpg,clothes
12408,shoes/shoes436.jpg,shoes
12409,clothes/clothes2765.jpg,clothes
12410,battery/battery874.jpg,battery


In [15]:
train_df['Category'].value_counts()

Category
clothes        4238
shoes          1596
paper           844
biological      804
battery         751
cardboard       717
plastic         685
white-glass     630
metal           601
trash           571
green-glass     498
brown-glass     477
Name: count, dtype: int64

In [16]:
validate_df['Category'].value_counts()

Category
clothes        548
shoes          200
biological     105
plastic         92
metal           89
battery         87
paper           86
white-glass     77
cardboard       76
brown-glass     67
green-glass     63
trash           61
Name: count, dtype: int64

In [17]:
test_df['Category'].value_counts()

Category
clothes        539
shoes          181
paper          120
battery        107
cardboard       98
plastic         88
metal           79
biological      76
white-glass     68
green-glass     68
trash           65
brown-glass     63
Name: count, dtype: int64

### TO CSV

In [18]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv',index=False)
validate_df.to_csv('validation.csv',index=False)

In [19]:
num_classes=12

In [20]:
img_height = 224
img_width = 224
img_size=(img_width,img_height)
batch_size = 32


In [21]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
data_directory = "garbage_classification/"

In [23]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory=data_directory, 
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 12412 validated image filenames belonging to 12 classes.


In [24]:
validation_generator = train_datagen.flow_from_dataframe(
    dataframe=validate_df,
    directory=data_directory,
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size)

Found 1551 validated image filenames belonging to 12 classes.


In [25]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=data_directory,
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 1552 validated image filenames belonging to 12 classes.


In [32]:
test_generator

In [26]:
from datetime import datetime
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def create_model(path, name, img_height, img_width, train_generator, validation_generator, epochs):
    num_classes = 12

    start = datetime.now()

    # Crear el modelo base
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Definir el modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    # Congelar las capas del modelo base
    for layer in base_model.layers:
        layer.trainable = False

    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

    try:
        model.save(f"{path}/{name}.h5")
    except:
        print("Modelo no guardado")

    end = datetime.now()

    print(f"It took {end-start} time")


In [27]:
create_model(".", "model_0306_efficientnetB0", 224, 224, train_generator, validation_generator, epochs=1)

388/388 [==============================] - 583s 1s/step - loss: 0.2323 - accuracy: 0.9272 - val_loss: 0.1748 - val_accuracy: 0.9491
It took 0:09:44.947151 time


In [28]:
from tensorflow.keras.models import load_model
model = load_model("model_0306_efficientnetB0.h5")

In [29]:
from tensorflow.keras.preprocessing import image

image_path = 'C:/Users/Leticia Martinez/Desktop/IRONHACK/Labs/Project_V/photo_test/plastic143.jpg'
img = image.load_img(image_path, target_size=(224, 224))


In [30]:
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalizar los valores de píxel entre 0 y 1
img_array = np.expand_dims(img_array, axis=0)  # Agregar una dimensión adicional para representar el batch


In [31]:
predictions = model.predict(img_array)

1/1 [==============================] - 1s 1s/step


In [32]:
predicted_class = np.argmax(predictions[0])
class_labels = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']

predicted_label = class_labels[predicted_class]
print('La imagen es:', predicted_label)

La imagen es: plastic


In [38]:
import numpy as np
from PIL import Image

# Ruta de la imagen en tu ordenador
image_path = r"C:\Users\Leticia Martinez\Desktop\IRONHACK\Labs\Project_V\photo_test\plastic3.jpg"

# Cargar la imagen en color
image = Image.open(image_path).convert('RGB')

# Redimensionar la imagen a las dimensiones requeridas por el modelo
img_width, img_height = 224, 224  # Usar las dimensiones adecuadas para tu modelo
image = image.resize((img_width, img_height))

# Convertir la imagen en un arreglo numpy
image_array = np.array(image)

# Añadir una dimensión adicional para el batch
image_array = np.expand_dims(image_array, axis=0)

# Preprocesar la imagen (normalización, etc.) de acuerdo con el modelo
image_array = image_array.astype('float32')
image_array /= 255

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)
confidence = np.max(predictions) * 100

print("La imagen es:", predicted_class)
print("Confianza (%):", confidence)

1/1 [==============================] - 0s 56ms/step
La imagen es: 8
Confianza (%): 22.18020111322403


In [39]:
import numpy as np
from PIL import Image

# Ruta de la imagen en tu ordenador
image_path = r"C:\Users\Leticia Martinez\Desktop\IRONHACK\Labs\Project_V\photo_test\clothes4.jpg"

# Cargar la imagen en color
image = Image.open(image_path).convert('RGB')

# Redimensionar la imagen a las dimensiones requeridas por el modelo
img_width, img_height = 224, 224  # Usar las dimensiones adecuadas para tu modelo
image = image.resize((img_width, img_height))

# Convertir la imagen en un arreglo numpy
image_array = np.array(image)

# Añadir una dimensión adicional para el batch
image_array = np.expand_dims(image_array, axis=0)

# Preprocesar la imagen (normalización, etc.) de acuerdo con el modelo
image_array = image_array.astype('float32')
image_array /= 255

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)
confidence = np.max(predictions) * 100

print("La imagen es:", predicted_class)
print("Confianza (%):", confidence)

1/1 [==============================] - 0s 44ms/step
La imagen es: 8
Confianza (%): 22.076191008090973


In [34]:
import numpy as np
from PIL import Image

# Ruta de la imagen en tu ordenador
image_path = r"C:\Users\Leticia Martinez\Desktop\IRONHACK\Labs\Project_V\photo_test\clothes4.jpg"

# Cargar la imagen en color
image = Image.open(image_path).convert('RGB')

# Redimensionar la imagen a las dimensiones requeridas por el modelo
img_width, img_height = 224, 224  # Usar las dimensiones adecuadas para tu modelo
image = image.resize((img_width, img_height))

# Convertir la imagen en un arreglo numpy
image_array = np.array(image)

# Añadir una dimensión adicional para el batch
image_array = np.expand_dims(image_array, axis=0)

# Preprocesar la imagen (normalización, etc.) de acuerdo con el modelo
image_array = image_array.astype('float32')
image_array /= 255

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions)

print("La imagen es:", predicted_class)

1/1 [==============================] - 0s 52ms/step
La imagen es: 8


In [35]:
class_labels = list(train_generator.class_indices.keys())
class_labels

['battery',
 'biological',
 'brown-glass',
 'cardboard',
 'clothes',
 'green-glass',
 'metal',
 'paper',
 'plastic',
 'shoes',
 'trash',
 'white-glass']

In [36]:
image_path = 'C:/Users/Leticia Martinez/Desktop/IRONHACK/Labs/Project_V/photo_test/metal5.jpg'
img = image.load_img(image_path, target_size=(224, 224))

AttributeError: load_img

In [57]:
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalizar los valores de píxel entre 0 y 1
img_array = np.expand_dims(img_array, axis=0)  # Agregar una dimensión adicional para representar el batch


In [58]:
predictions = model.predict(img_array)

1/1 [==============================] - 0s 169ms/step


In [59]:
predicted_class = np.argmax(predictions[0])
class_labels = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']

predicted_label = class_labels[predicted_class]
print('La imagen es:', predicted_label)

La imagen es: plastic
